In [1]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter, ExcelFile
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import string
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from collections import defaultdict
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /home/veera/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/veera/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/veera/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#database connection using python
import mysql.connector as mc
from mysql.connector import Error
from mysql.connector import MySQLConnection

def connect():
    """connect to mysql database"""
    conn = None
    try:
        conn = mc.connect(host='localhost', database='abusive_words', user='root', password='password')
        if conn.is_connected():
            print('Connected to MySQL database')
        return conn
    except Error as e:
        print(e)
    #finally :
     #   if conn is not None and conn.is_connected():
      #      conn.close()
       #     print('connection closed')
        
def query_with_fetchone(conn):
    
    try:
        cursor = conn.cursor()
        cursor.execute("select * from abusive LIMIT 10;")
        row = cursor.fetchone()
        #print(cursor.fetchone())
        while row is not None:
            print(row)
            row = cursor.fetchone()
    except Error as e:
        print(e)
    finally : 
        cursor.close()
        conn.close()

#a generator used to fetch queries size at a time, size can be 1, .... any number
#present no need to use generator 
def iter_row(cursor, size=1):
    while True:
        rows = cursor.fetchmany(size)
        if not rows:
            break
        for row in rows:
            yield row
            
        
def check_the_word(conn, wrd):
    
    if len(wrd) > 3:
        wrd = wrd[:len(wrd)-1]
        
    try:
        query = "select * from abusive where abusive_wrds like \'"+wrd+"%\';"
        cursor = conn.cursor()
        cursor.execute(query)
        row = cursor.fetchone()
        #print(row)
        if row is None:
            return 0, None
        else:
            ls = [wrd]
            #ls.extend(list(row))
            while row is not None:
                ls.append(row)
                row = cursor.fetchone()
            
            return 1, ls
        
    except Error as e:
        print(e)

#func for checking list on words in database
def check_list(conn, wds):
    try:
        qry = "select distinct * from abusive where"
        for w in wds:
            qry += " abusive_wrds like \'"+w+"%\' or"
        qry = qry[:-3]
        qry += ";"
        print(qry)
        cursor = conn.cursor()
        cursor.execute(qry)
        row = cursor.fetchall()
        if row is None:
            return 0, []
        else:
            return 1, [row] 
        
    except Error as e:
        print(e)
    
    
def close_the_connection(conn):
    try:
        conn.close()
        print("connection is closed")
    except Error as e:
        print(e)

        
            
if __name__ == '__main__':
    c = connect()
    #query_with_fetchone(c)

Connected to MySQL database


In [3]:
#functions for above process
def remove_punct(text):
    s =set(['"','(',')','.',',','-','<','>','/','\',%', '\\x', '!','?']) #set(string.punctuation)
    #s.remove("'")
    #s = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'
    no_pnt = "".join([c for c in text if c not in s])
    return no_pnt

def tknz_text(text):
    #lis = re.split('\W+', text)
    #return lis
    #or
    return word_tokenize(text)
    
#after tokenization, we have to do remove stop words
def remove_stopwords(text):
    stopword = nltk.corpus.stopwords.words('english')
    text = [word for word in text if word not in stopword]
    #text.append()
    return text

#func for stemming --- here we use snowball stemmer over porterstemmer and lancaster stemmer
def stmng(wrds):
    stemmer = SnowballStemmer("english")
    res = []
    for w in wrds:
        res.append(stemmer.stem(w))
    return res 
#wordnet lemmatizer
def lem_without(wrds):
    lemmatizer = WordNetLemmatizer()
    r = []
    for w in wrds:
        r.append(lemmatizer.lemmatize(w))
    return r
#wordnet lemmatizer with pos_tagging
def lem_with(wrds, dic_map):
    lemma = WordNetLemmatizer()
    r = []
    for wd, tag in nltk.pos_tag(wrds):
        r.append(lemma.lemmatize(wd, dic_map[tag[0]]))
    return r



In [3]:
import time

In [4]:
#example for one sentence, similar we can do it for entire dataset by loading into pandas dataframe
#below are some examples for abusive and normal text
#s = "ok but where the hell was it released?you all copy articles from each pther but did anyone do the fucking search to tell us where the fuck was it released?fuck it."
#s = "Are you kidding me?"
#s = "You're a crank."
#%%timeit
#print(start_time)
start_time = time.time()
s = "Fucking Red Neck Bitch!! who"
temp = remove_punct(s)
print(temp)
temp = tknz_text(temp)
print(temp)
temp = remove_stopwords(temp)
print(temp)
temp = stmng(temp) #lem_with(temp, dic_map)#stmng(temp)
print(temp)
c = connect()
#checking each word in our database
flag = 0

'''for wd in temp:
    f, word = check_the_word(c, wd)
    if flag  == 0 and f:
        flag = 1
        break
if flag == 1:
    print("Abusive Detected")
    r = []
    for w in word:
        w = str(w)
        w = w.replace("('","")
        w = w.replace(",)", "")
        r.append(w)
    print("search word --------",r[0])
    print("words are ",r)
else:
    print("Normal Text")'''

fg, word = check_list(c, temp)
if fg == 1:
    print("Abusive Detected :", word)
else:
    print("Normal Text")
    
    
    
close_the_connection(c)
print()
print(time.time(), "-", start_time)
print(time.time()-start_time)
    

NameError: name 'remove_punct' is not defined